In [10]:
import tushare as ts

pro = ts.pro_api()

In [11]:
df = pro.daily_basic()
securities = df["ts_code"].tolist()
securities[:5]

2025-09-22 18:14:23.082 | INFO     | tushare:_connect:29 - Connected to server tushare:5290


['603811.SH', '301539.SZ', '002335.SZ', '002516.SZ', '300016.SZ']

In [13]:
import time
dfs = []

start = datetime.date(2018, 1, 1)
end = datetime.date(2023, 12, 31)
for i, sec in enumerate(securities):
    df = pro.fina_audit(ts_code = sec, 
                        start_date = start.strftime("%Y%m%d"), 
                        end_date=end.strftime("%Y%m%d"))
    if not df.empty:
        dfs.append(df)

    time.sleep(0.125)
    if i % 500 == 0:
        print(f"processed {i} securities")

audit_report = pd.concat(dfs, axis=0)
audit_report.head()

processed 0 securities
processed 500 securities
processed 1000 securities
processed 1500 securities
processed 2000 securities
processed 2500 securities
processed 3000 securities
processed 3500 securities
processed 4000 securities
processed 4500 securities


2025-09-22 18:56:55.998 | ERROR    | tushare:_receive_data:125 - Error receiving data: timed out
2025-09-22 18:56:56.002 | WARNING  | tushare:_send_request:78 - Connection failed (attempt 1/4): timed out
2025-09-22 18:56:56.003 | INFO     | tushare:_send_request:79 - Retrying in 1 seconds...


NameError: name 'time' is not defined

In [16]:
temp = pd.concat(dfs)
len(temp["ts_code"].unique())

4809

In [20]:
rest = set(securities) - set(temp["ts_code"].unique())
for i, sec in enumerate(rest):
    df = pro.fina_audit(ts_code = sec, 
                        start_date = start.strftime("%Y%m%d"), 
                        end_date=end.strftime("%Y%m%d"))
    if not df.empty:
        dfs.append(df)

audit_report = pd.concat(dfs, axis=0)
audit_report.head()

2025-09-22 18:59:31.552 | INFO     | tushare:_connect:29 - Connected to server tushare:5290


,ts_code,ann_date,end_date,audit_result,audit_fees,audit_agency,audit_sign
0,603811.SH,20230426,20221231,标准无保留意见,1000000.0,中汇会计师事务所,"翟晓宁,曹吉诚"
1,603811.SH,20220426,20211231,标准无保留意见,1000000.0,中汇会计师事务所,"翟晓宁,曹吉诚"
2,603811.SH,20210423,20201231,标准无保留意见,1000000.0,中汇会计师事务所,"鲁立,曹吉诚"
3,603811.SH,20200428,20191231,标准无保留意见,1000000.0,中汇会计师事务所,"鲁立,邢兵"
4,603811.SH,20190410,20181231,标准无保留意见,550000.0,中汇会计师事务所,"鲁立,翟晓宁"


In [ ]:

audit_report.query("ts_code == '000004.SZ'")

,ts_code,ann_date,end_date,audit_result,audit_fees,audit_agency,audit_sign
0,000004.SZ,20230429,20221231,标准无保留意见,1200000.0,中兴财光华会计师事务所,"侯胜利,杜丽"
1,000004.SZ,20220430,20211231,保留意见,1200000.0,致同会计师事务所,"付细军,庄翠曼"
2,000004.SZ,20210428,20201231,标准无保留意见,800000.0,大华会计师事务所,"申宏波,万海青"
3,000004.SZ,20200428,20191231,标准无保留意见,900000.0,大华会计师事务所,"申宏波,赵君"
4,000004.SZ,20190426,20181231,标准无保留意见,380000.0,大华会计师事务所,"刘金平,赵君"
5,000004.SZ,20180428,20171231,带强调事项段的无保留意见,380000.0,大华会计师事务所,"胡进科,赵君"


In [ ]:
import datetime
import pandas as pd
import tushare as ts
from helper import ParquetUnifiedStorage

def fetch_fina_audit(start: datetime.date, end: datetime.date) -> pd.DataFrame | None:
    """
    通过 tushare 接口，获取财务审计意见数据。该函数通过遍历指定日期范围内的每一天来获取数据。

    Args:
        start: 开始日期 (基于公告日期 ann_date)
        end: 结束日期 (基于公告日期 ann_date)

    Returns:
        DataFrame: 包含审计意见等数据的DataFrame，如果无数据则返回None
    """
    pro = ts.pro_api()
    all_data = []

    df = pro.daily_basic()
    securities = df["ts_code"].tolist()

    for sec in securities:
        try:
            df = pro.fina_audit(ts_code = sec, 
                            start_date = start.strftime("%Y%m%d"), 
                            end_date=end.strftime("%Y%m%d"))
            if not df.empty:
                all_data.append(df)
        except Exception as e:
            logger.error(f"获取 {sec} 的财务审计意见失败: {e}")
            continue

    if len(all_data) == 0:
        logger.warning(f"在 {start} 到 {end} 之间，没有获取到任何财务审计意见数据")
        return None
        
    df = pd.concat(all_data, ignore_index=True)
    df = df.rename(columns={'end_date': 'date', 'ts_code': 'asset'})

    df['date'] = pd.to_datetime(df['date'], format='%Y%m%d').dt.date
    
    return df

from helper import ParquetUnifiedStorage
fina_audit_storage_path = data_home / "rw/fina_audit.parquet"

audit_storage = ParquetUnifiedStorage(
    store_path=fina_audit_storage_path,
    fetch_data_func=fetch_fina_audit
)

df = pd.read_parquet("/tmp/audit_report.parquet")
audit_storage.append_data(df)

start = datetime.date(2018, 1, 1)
end = datetime.date(2023, 12, 31)

audit_storage.load_data(start, end)

2025-09-22 20:43:54.166 | INFO     | helper:load_data:406 - 获取结束日期后缺失的数据: 2023-10-01 到 2023-12-31
2025-09-22 20:43:54.168 | INFO     | tushare:_connect:29 - Connected to server tushare:5290


In [1]:
from helper import ParquetUnifiedStorage
fina_audit_storage_path = data_home / "rw/fina_audit.parquet"

audit_storage = ParquetUnifiedStorage(
    store_path=fina_audit_storage_path,
    fetch_data_func=fetch_fina_audit
)

df = pd.read_parquet("/tmp/audit_report.parquet")
audit_storage.append_data(df)
audit_storage.load_data(start, end)

NameError: name 'fetch_fina_audit' is not defined

In [39]:
df = audit_report.rename(columns={'end_date': 'date', 'ts_code': 'asset'})
df.to_parquet('/tmp/audit_report.parquet', index=False)
